In [0]:
%fs ls "/FinalProject"

path,name,size,modificationTime
dbfs:/FinalProject/All_Candidates_Cleaned/,All_Candidates_Cleaned/,0,0
dbfs:/FinalProject/Cont_by_indv/,Cont_by_indv/,0,0
dbfs:/FinalProject/PAC/,PAC/,0,0
dbfs:/FinalProject/all/,all/,0,0
dbfs:/FinalProject/cand_master/,cand_master/,0,0
dbfs:/FinalProject/ccl/,ccl/,0,0
dbfs:/FinalProject/cm/,cm/,0,0
dbfs:/FinalProject/cmast/,cmast/,0,0
dbfs:/FinalProject/cmt_mast/,cmt_mast/,0,0
dbfs:/FinalProject/cont_and_ind_exp/,cont_and_ind_exp/,0,0


####All Candidates File

In [0]:
# Reading All Candiates parquet file for cleaning 
All_cand ="/FinalProject/all"
All_candidates = (spark
      .read
      .format("parquet")
      .option("path", All_cand)
      .load())

**Creating a method "shape" to show number of rows and columns for each file**

In [0]:
import pyspark
def Shape(dataFrame):
    return (dataFrame.count(), len(dataFrame.columns))
pyspark.sql.dataframe.DataFrame.shape = Shape
print(All_candidates.shape())

(3981, 30)


In [0]:
# Distinct checks whether the dataframe has duplicate values
print("distinct count: " + str(All_candidates.distinct().count()))

distinct count: 3981


In [0]:
from pyspark.sql.functions import col,isnan, when, count

All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[:10]]).show()
All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[10:20]]).show()
All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[20:30]]).show()

+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|CAND_ID|CAND_NAME|CAND_ICI|PTY_CD|CAND_PTY_AFFILIATION|TTL_RECEIPTS|TRANS_FROM_AUTH|TTL_DISB|TRANS_TO_AUTH|COH_BOP|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|      0|        0|      78|     0|                   1|           0|              0|       0|            0|      0|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+

+-------+------------+----------+-----------+---------------+----------------+-------------+-----------------+--------------+--------------------+
|COH_COP|CAND_CONTRIB|CAND_LOANS|OTHER_LOANS|CAND_LOAN_REPAY|OTHER_LOAN_REPAY|DEBTS_OWED_BY|TTL_INDIV_CONTRIB|CAND_OFFICE_ST|CAND_OFFICE_DISTRICT|
+-------+------------+----------+-----------+---------------+----------------+-------------+-----------------+----------

In [0]:
# Dropping the columns which have null values
null_columns=("SPEC_ELECTION","PRIM_ELECTION","RUN_ELECTION","GEN_ELECTION","GEN_ELECTION_PRECENT")
All_candidates=All_candidates.drop(*null_columns)
len(All_candidates.columns)

Out[5]: 25

In [0]:
display(All_candidates.describe())

summary,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
count,3981,3981,3903,3981,3980,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981,3981
mean,null,null,null,1.6541070082893745,null,3490903.2603391143,199196.3327103742,3182462.0778698847,21569.690022607374,160973.92352172834,2696521.8314318014,1110614.6236347647,84475.35304948506,3220.5752851042453,509985.2855212257,14861.308646068828,62000.00327304696,6174937.332265763,0.0,8.351921627731725,614102.8374328063,1773.4558301934187,null,510921.42914594355,1284.31341873901
stddev,null,null,null,0.6416658107355816,null,8.233354043721126E7,5849210.709572181,6.819973962700449E7,326093.2548138953,985554.3443694386,1.5464019963747975E8,4.8379044930261254E7,1152036.1495016995,128069.37137991715,3.0488734755703248E7,796930.105425312,825966.27195719,2.9935798304538614E8,0.0,10.60278665607577,3.0485939003770217E7,49372.89487688053,null,3.0487148164347306E7,8512.58424575805
min,H0AL01055,ANGELA GLASS,C,1,AMP,-674132.5,0.0,-674157.5,0.0,-21673.46,-380765.96,0.0,0.0,0.0,0.0,0.0,-15000.0,-900.0,00,00,0.0,0.0,01/01/2019,-37410.0,-20400.0
max,S8WV00143,"ZUNKER, TRICIA",O,3,WFP,4.824617973E9,2.7517652812E8,3.848951085E9,1.0415E7,3.0E7,9.7566690782E9,2.831281203E9,5.001902222E7,7300000.0,1.9235003E9,4.998711567E7,5.00186604E7,1.8853982587E10,WY,53,1.9235003E9,3100000.0,12/31/2020,1.9235003E9,224100.0


In [0]:
from pyspark.sql.functions import when, lit

In [0]:
TTL_RECEIPTS_correct = when(((All_candidates['TRANS_FROM_AUTH'] != 0.0) & (All_candidates['TRANS_TO_AUTH'] != 0.0)), ((All_candidates['TTL_RECEIPTS'] - All_candidates['TRANS_FROM_AUTH'])))
All_candidates = All_candidates.withColumn('TTL_RECEIPTS_CORRECTED', TTL_RECEIPTS_correct)
All_candidates.shape()

Out[8]: (3981, 26)

In [0]:
All_candidates.printSchema()

root
 |-- CAND_ID: string (nullable = true)
 |-- CAND_NAME: string (nullable = true)
 |-- CAND_ICI: string (nullable = true)
 |-- PTY_CD: integer (nullable = true)
 |-- CAND_PTY_AFFILIATION: string (nullable = true)
 |-- TTL_RECEIPTS: double (nullable = true)
 |-- TRANS_FROM_AUTH: double (nullable = true)
 |-- TTL_DISB: double (nullable = true)
 |-- TRANS_TO_AUTH: double (nullable = true)
 |-- COH_BOP: double (nullable = true)
 |-- COH_COP: double (nullable = true)
 |-- CAND_CONTRIB: double (nullable = true)
 |-- CAND_LOANS: double (nullable = true)
 |-- OTHER_LOANS: double (nullable = true)
 |-- CAND_LOAN_REPAY: double (nullable = true)
 |-- OTHER_LOAN_REPAY: double (nullable = true)
 |-- DEBTS_OWED_BY: double (nullable = true)
 |-- TTL_INDIV_CONTRIB: double (nullable = true)
 |-- CAND_OFFICE_ST: string (nullable = true)
 |-- CAND_OFFICE_DISTRICT: string (nullable = true)
 |-- OTHER_POL_CMTE_CONTRIB: double (nullable = true)
 |-- POL_PTY_CONTRIB: double (nullable = true)
 |-- CVG_END_

In [0]:
All_candidates.select('TRANS_FROM_AUTH').show()

+---------------+
|TRANS_FROM_AUTH|
+---------------+
|      138304.94|
|            0.0|
|       60024.76|
|            0.0|
|            0.0|
|            0.0|
|      102070.05|
|            0.0|
|            0.0|
|            0.0|
|            0.0|
|       19259.14|
|          11.32|
|            0.0|
|            0.0|
|            0.0|
|            0.0|
|            0.0|
|            0.0|
|         3351.7|
+---------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import coalesce
All_candidates=All_candidates.withColumn('TTL_RECEIPTS_CORRECTED',coalesce(All_candidates.TTL_RECEIPTS_CORRECTED,All_candidates.TTL_RECEIPTS))                                                                         

In [0]:
All_candidates.select('TTL_RECEIPTS_CORRECTED').show()

+----------------------+
|TTL_RECEIPTS_CORRECTED|
+----------------------+
|            1950289.86|
|                   0.0|
|            5253251.54|
|              80094.95|
|             118661.85|
|              62935.42|
|            2344516.53|
|             182564.96|
|              491715.7|
|            1323944.09|
|              26809.68|
|              525384.2|
|              57722.77|
|               12230.0|
|              216297.5|
|               15043.0|
|            2615977.24|
|             286727.66|
|               22950.0|
|             476139.02|
+----------------------+
only showing top 20 rows



In [0]:
TTL_DISB_correct = when(((All_candidates['TRANS_FROM_AUTH'] != 0.0) & (All_candidates['TRANS_TO_AUTH'] != 0.0)), ((All_candidates['TTL_DISB'] - All_candidates['TRANS_TO_AUTH'])))
All_candidates = All_candidates.withColumn('TTL_DISB_CORRECTED', TTL_DISB_correct)

In [0]:
All_candidates=All_candidates.withColumn('TTL_DISB_CORRECTED',coalesce(All_candidates.TTL_DISB_CORRECTED,All_candidates.TTL_DISB))

In [0]:
All_candidates.select("CAND_ICI").distinct().show()

+--------+
|CAND_ICI|
+--------+
|    null|
|       O|
|       C|
|       I|
+--------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

All_candidates=All_candidates.withColumn('CAND_ICI', 
      when((All_candidates.CAND_ICI == 'C'),regexp_replace(All_candidates.CAND_ICI,'C','Challenger')) \
     .when((All_candidates.CAND_ICI == 'I'),regexp_replace(All_candidates.CAND_ICI,'I','Incumbent')) \
     .when((All_candidates.CAND_ICI == 'O'),regexp_replace(All_candidates.CAND_ICI,'O','Open Seat')))

In [0]:
All_candidates=All_candidates.na.fill("Unkown")
All_candidates.select("CAND_ICI").distinct().show()
All_candidates.shape()

+----------+
|  CAND_ICI|
+----------+
| Open Seat|
|    Unkown|
|Challenger|
| Incumbent|
+----------+

Out[17]: (3981, 27)

In [0]:
All_candidates.select("CAND_OFFICE_DISTRICT").take(10)

Out[18]: [Row(CAND_OFFICE_DISTRICT='00'),
 Row(CAND_OFFICE_DISTRICT='00'),
 Row(CAND_OFFICE_DISTRICT='00'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01'),
 Row(CAND_OFFICE_DISTRICT='01')]

In [0]:
All_candidates.write.option("header",True) \
        .partitionBy("CAND_ICI") \
        .mode("overwrite") \
        .parquet("/FinalProject/All_Candidates_Cleaned")

In [0]:
from pyspark.sql.functions import col,isnan, when, count

All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[:10]]).show()
All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[10:20]]).show()
All_candidates.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in All_candidates.columns[20:30]]).show()

+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|CAND_ID|CAND_NAME|CAND_ICI|PTY_CD|CAND_PTY_AFFILIATION|TTL_RECEIPTS|TRANS_FROM_AUTH|TTL_DISB|TRANS_TO_AUTH|COH_BOP|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|      0|        0|       0|     0|                   0|           0|              0|       0|            0|      0|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+

+-------+------------+----------+-----------+---------------+----------------+-------------+-----------------+--------------+--------------------+
|COH_COP|CAND_CONTRIB|CAND_LOANS|OTHER_LOANS|CAND_LOAN_REPAY|OTHER_LOAN_REPAY|DEBTS_OWED_BY|TTL_INDIV_CONTRIB|CAND_OFFICE_ST|CAND_OFFICE_DISTRICT|
+-------+------------+----------+-----------+---------------+----------------+-------------+-----------------+----------

####PAC

In [0]:
PAC ="dbfs:/FinalProject/PAC/"
PAC_file = (spark
      .read
      .format("parquet")
      .option("path", PAC)
      .load())

In [0]:
PAC_file.shape()

Out[22]: (11534, 27)

In [0]:
from pyspark.sql.functions import col,isnan, when, count

PAC_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in PAC_file.columns[:9]]).show()
PAC_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in PAC_file.columns[10:19]]).show()
PAC_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in PAC_file.columns[20:27]]).show()

+-------+-------+-------+---------+----------------+------------+--------------+------------+----------------------+
|CMTE_ID|CMTE_NM|CMTE_TP|CMTE_DSGN|CMTE_FILING_FREQ|TTL_RECEIPTS|TRANS_FROM_AFF|INDV_CONTRIB|OTHER_POL_CMTE_CONTRIB|
+-------+-------+-------+---------+----------------+------------+--------------+------------+----------------------+
|      0|      0|      0|        1|               0|        2066|          2066|        2066|                  2066|
+-------+-------+-------+---------+----------------+------------+--------------+------------+----------------------+

+----------+------------------+--------+------------+------------+----------------------+---------------+----------+-------+
|CAND_LOANS|TTL_LOANS_RECEIVED|TTL_DISB|TRANF_TO_AFF|INDV_REFUNDS|OTHER_POL_CMTE_REFUNDS|CAND_LOAN_REPAY|LOAN_REPAY|COH_BOP|
+----------+------------------+--------+------------+------------+----------------------+---------------+----------+-------+
|      2066|              2066|    2066

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

PAC_file=PAC_file.withColumn('CMTE_DSGN', 
      when((PAC_file.CMTE_DSGN == 'A'),regexp_replace(PAC_file.CMTE_DSGN,'A','Authorized by a candidate')) \
     .when((PAC_file.CMTE_DSGN == 'B'),regexp_replace(PAC_file.CMTE_DSGN,'B','Lobbyist/Registrant PAC')) \
     .when((PAC_file.CMTE_DSGN == 'D'),regexp_replace(PAC_file.CMTE_DSGN,'D','Leadership PAC')) \
     .when((PAC_file.CMTE_DSGN == 'J'),regexp_replace(PAC_file.CMTE_DSGN,'J','Joint fundraiser')) \
     .when((PAC_file.CMTE_DSGN == 'P'),regexp_replace(PAC_file.CMTE_DSGN,'P','Principal campaign committee of a candidate')) \
     .when((PAC_file.CMTE_DSGN == 'U'),regexp_replace(PAC_file.CMTE_DSGN,'U','Unauthorized'))) 

PAC_file.select("CMTE_DSGN").distinct().show()

+--------------------+
|           CMTE_DSGN|
+--------------------+
|        Unauthorized|
|Lobbyist/Registra...|
|                null|
|      Leadership PAC|
|Principal campaig...|
|    Joint fundraiser|
+--------------------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

PAC_file=PAC_file.withColumn('CMTE_FILING_FREQ', 
      when((PAC_file.CMTE_FILING_FREQ == 'A'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'A','Administratively terminated')) \
     .when((PAC_file.CMTE_FILING_FREQ == 'D'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'D','Debt')) \
     .when((PAC_file.CMTE_FILING_FREQ == 'M'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'M','Monthly filer')) \
     .when((PAC_file.CMTE_FILING_FREQ == 'Q'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'Q','Quarterly filer')) \
     .when((PAC_file.CMTE_FILING_FREQ == 'T'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'T','Terminated')) \
     .when((PAC_file.CMTE_FILING_FREQ == 'W'),regexp_replace(PAC_file.CMTE_FILING_FREQ,'W','Waived'))) 

PAC_file.select("CMTE_FILING_FREQ").distinct().show()

+--------------------+
|    CMTE_FILING_FREQ|
+--------------------+
|       Monthly filer|
|          Terminated|
|     Quarterly filer|
|Administratively ...|
+--------------------+



In [0]:
PAC_file.write.option("header",True) \
        .partitionBy("CMTE_FILING_FREQ") \
        .mode("overwrite") \
        .parquet("/FinalProject/PAC_Cleaned")

####Candidate Master

In [0]:
cand_master ="dbfs:/FinalProject/cand_master/"
cand_master_file = (spark
      .read
      .format("parquet")
      .option("path", cand_master)
      .load())

In [0]:
cand_master_file.shape()

Out[29]: (7770, 15)

In [0]:
from pyspark.sql.functions import col,isnan, when, count

cand_master_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cand_master_file.columns[:5]]).show()
cand_master_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cand_master_file.columns[5:10]]).show()
cand_master_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cand_master_file.columns[10:15]]).show()

+-------+---------+--------------------+----------------+--------------+
|CAND_ID|CAND_NAME|CAND_PTY_AFFILIATION|CAND_ELECTION_YR|CAND_OFFICE_ST|
+-------+---------+--------------------+----------------+--------------+
|      0|        0|                   5|               0|             0|
+-------+---------+--------------------+----------------+--------------+

+-----------+--------------------+--------+-----------+--------+
|CAND_OFFICE|CAND_OFFICE_DISTRICT|CAND_ICI|CAND_STATUS|CAND_PCC|
+-----------+--------------------+--------+-----------+--------+
|          0|                   7|     324|          0|    1424|
+-----------+--------------------+--------+-----------+--------+

+--------+--------+---------+-------+--------+
|CAND_ST1|CAND_ST2|CAND_CITY|CAND_ST|CAND_ZIP|
+--------+--------+---------+-------+--------+
|     188|    6853|        3|    179|     190|
+--------+--------+---------+-------+--------+



In [0]:
cand_master_file=cand_master_file.drop("CAND_ST2")
len(cand_master_file.columns)

Out[31]: 14

In [0]:
cand_master_file = cand_master_file.withColumn("CAND_ZIP", cand_master_file["CAND_ZIP"].cast('int'))
cand_master_file.schema["CAND_ZIP"].dataType

Out[32]: IntegerType

In [0]:
from pyspark.sql.functions import *
cand_master_file = cand_master_file.withColumn('CAND_ZIP', lpad(cand_master_file.CAND_ZIP,5, '0'))
cand_master_file.select("CAND_ZIP").distinct().tail(5)

Out[33]: [Row(CAND_ZIP='23185'),
 Row(CAND_ZIP='30809'),
 Row(CAND_ZIP='70898'),
 Row(CAND_ZIP='82716'),
 Row(CAND_ZIP='39039')]

In [0]:
cand_master_file = cand_master_file.withColumn("CAND_OFFICE_DISTRICT", cand_master_file["CAND_OFFICE_DISTRICT"].cast('int'))

In [0]:
from pyspark.sql.functions import *
cand_master_file = cand_master_file.withColumn('CAND_OFFICE_DISTRICT', lpad(cand_master_file.CAND_OFFICE_DISTRICT,2, '0'))
cand_master_file.select("CAND_OFFICE_DISTRICT").distinct().tail(3)

Out[35]: [Row(CAND_OFFICE_DISTRICT='80'),
 Row(CAND_OFFICE_DISTRICT='50'),
 Row(CAND_OFFICE_DISTRICT='45')]

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
cand_master_file=cand_master_file.withColumn('CAND_OFFICE', 
when((cand_master_file.CAND_OFFICE == 'H'),regexp_replace(cand_master_file.CAND_OFFICE,'H','House')) \
.when((cand_master_file.CAND_OFFICE == 'S'),regexp_replace(cand_master_file.CAND_OFFICE,'S','Senate')) \
.when((cand_master_file.CAND_OFFICE == 'P'),regexp_replace(cand_master_file.CAND_OFFICE,'P','President'))) 
cand_master_file.select("CAND_OFFICE").distinct().show()

+-----------+
|CAND_OFFICE|
+-----------+
|     Senate|
|  President|
|      House|
+-----------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
cand_master_file=cand_master_file.withColumn('CAND_STATUS', 
      when((cand_master_file.CAND_STATUS == 'C'),regexp_replace(cand_master_file.CAND_STATUS,'C','Statutory candidate')) \
     .when((cand_master_file.CAND_STATUS == 'F'),regexp_replace(cand_master_file.CAND_STATUS,'F','Statutory candidate for future election')) \
     .when((cand_master_file.CAND_STATUS == 'N'),regexp_replace(cand_master_file.CAND_STATUS,'N','Not yet a statutory candidate')) \
     .when((cand_master_file.CAND_STATUS == 'P'),regexp_replace(cand_master_file.CAND_STATUS,'P','Statutory candidate in prior cycle'))) 
cand_master_file.select("CAND_STATUS").distinct().show()

+--------------------+
|         CAND_STATUS|
+--------------------+
|Statutory candida...|
| Statutory candidate|
|Not yet a statuto...|
|Statutory candida...|
+--------------------+



In [0]:
cand_master_file.write.option("header",True) \
        .partitionBy("CAND_STATUS") \
        .mode("overwrite") \
        .parquet("/FinalProject/Candidate_Master_Cleaned")

####Candidate Committee Linkage

In [0]:
CCL ="dbfs:/FinalProject/ccl/"
CCL_file = (spark
      .read
      .format("parquet")
      .option("path", CCL)
      .load())

In [0]:
CCL_file.shape()

Out[40]: (7060, 7)

In [0]:
from pyspark.sql.functions import col,isnan, when, count

CCL_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in CCL_file.columns[:7]]).show()

+-------+----------------+---------------+-------+-------+---------+----------+
|CAND_ID|CAND_ELECTION_YR|FEC_ELECTION_YR|CMTE_ID|CMTE_TP|CMTE_DSGN|LINKAGE_ID|
+-------+----------------+---------------+-------+-------+---------+----------+
|      0|               0|              0|      0|      0|        0|         0|
+-------+----------------+---------------+-------+-------+---------+----------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
CCL_file=CCL_file.withColumn('CMTE_DSGN', 
      when((CCL_file.CMTE_DSGN == 'A'),regexp_replace(CCL_file.CMTE_DSGN,'A','Authorized by a candidate')) \
     .when((CCL_file.CMTE_DSGN == 'B'),regexp_replace(CCL_file.CMTE_DSGN,'B','Lobbyist/Registrant PAC')) \
     .when((CCL_file.CMTE_DSGN == 'D'),regexp_replace(CCL_file.CMTE_DSGN,'D','Leadership PAC')) \
     .when((CCL_file.CMTE_DSGN == 'J'),regexp_replace(CCL_file.CMTE_DSGN,'J','Joint fundraiser')) \
     .when((CCL_file.CMTE_DSGN == 'P'),regexp_replace(CCL_file.CMTE_DSGN,'P','Principal campaign committee of a candidate')) \
     .when((CCL_file.CMTE_DSGN == 'U'),regexp_replace(CCL_file.CMTE_DSGN,'U','Unauthorized'))) 
CCL_file.select("CMTE_DSGN").distinct().show()

+--------------------+
|           CMTE_DSGN|
+--------------------+
|        Unauthorized|
|Lobbyist/Registra...|
|Authorized by a c...|
|Principal campaig...|
|    Joint fundraiser|
+--------------------+



In [0]:
CCL_file.where(CCL_file.CMTE_DSGN.contains('candidate')).count()

Out[43]: 6626

In [0]:
CCL_file.write.option("header",True) \
.mode("overwrite") \
.parquet("/FinalProject/CCL_file_Cleaned")

####Committee Master

In [0]:
committe_master ="dbfs:/FinalProject/cmt_mast/"
cmt_mast_file = (spark
      .read
      .format("parquet")
      .option("path", committe_master)
      .load())

In [0]:
cmt_mast_file.shape()

Out[46]: (18288, 15)

In [0]:
from pyspark.sql.functions import col,isnan, when, count
cmt_mast_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cmt_mast_file.columns[:5]]).show()
cmt_mast_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cmt_mast_file.columns[5:10]]).show()
cmt_mast_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cmt_mast_file.columns[10:15]]).show()

+-------+-------+-------+--------+--------+
|CMTE_ID|CMTE_NM|TRES_NM|CMTE_ST1|CMTE_ST2|
+-------+-------+-------+--------+--------+
|      0|      2|    729|       8|   13768|
+-------+-------+-------+--------+--------+

+---------+-------+--------+---------+-------+
|CMTE_CITY|CMTE_ST|CMTE_ZIP|CMTE_DSGN|CMTE_TP|
+---------+-------+--------+---------+-------+
|        9|      8|      13|        5|      5|
+---------+-------+--------+---------+-------+

+--------------------+----------------+------+----------------+-------+
|CMTE_PTY_AFFILIATION|CMTE_FILING_FREQ|ORG_TP|CONNECTED_ORG_NM|CAND_ID|
+--------------------+----------------+------+----------------+-------+
|               10484|               0| 15039|            9004|  11557|
+--------------------+----------------+------+----------------+-------+



In [0]:
cmt_mast_file=cmt_mast_file.drop("CMTE_ST2")
len(cmt_mast_file.columns)

Out[48]: 14

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cmt_mast_file=cmt_mast_file.withColumn('CMTE_DSGN', 
      when((cmt_mast_file.CMTE_DSGN == 'A'),regexp_replace(cmt_mast_file.CMTE_DSGN,'A','Authorized by a candidate')) \
     .when((cmt_mast_file.CMTE_DSGN == 'B'),regexp_replace(cmt_mast_file.CMTE_DSGN,'B','Lobbyist/Registrant PAC')) \
     .when((cmt_mast_file.CMTE_DSGN == 'D'),regexp_replace(cmt_mast_file.CMTE_DSGN,'D','Leadership PAC')) \
     .when((cmt_mast_file.CMTE_DSGN == 'J'),regexp_replace(cmt_mast_file.CMTE_DSGN,'J','Joint fundraiser')) \
     .when((cmt_mast_file.CMTE_DSGN == 'P'),regexp_replace(cmt_mast_file.CMTE_DSGN,'P','Principal campaign committee of a candidate')) \
     .when((cmt_mast_file.CMTE_DSGN == 'U'),regexp_replace(cmt_mast_file.CMTE_DSGN,'U','Unauthorized'))) 

cmt_mast_file.select("CMTE_DSGN").distinct().show()

+--------------------+
|           CMTE_DSGN|
+--------------------+
|        Unauthorized|
|Lobbyist/Registra...|
|                null|
|      Leadership PAC|
|Authorized by a c...|
|Principal campaig...|
|    Joint fundraiser|
+--------------------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cmt_mast_file=cmt_mast_file.withColumn('CMTE_FILING_FREQ', 
      when((cmt_mast_file.CMTE_FILING_FREQ == 'A'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'A','Administratively terminated')) \
     .when((cmt_mast_file.CMTE_FILING_FREQ == 'D'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'D','Debt')) \
     .when((cmt_mast_file.CMTE_FILING_FREQ == 'M'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'M','Monthly filer')) \
     .when((cmt_mast_file.CMTE_FILING_FREQ == 'Q'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'Q','Quarterly filer')) \
     .when((cmt_mast_file.CMTE_FILING_FREQ == 'T'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'T','Terminated')) \
     .when((cmt_mast_file.CMTE_FILING_FREQ == 'W'),regexp_replace(cmt_mast_file.CMTE_FILING_FREQ,'W','Waived'))) 

cmt_mast_file.select("CMTE_FILING_FREQ").distinct().show()

+--------------------+
|    CMTE_FILING_FREQ|
+--------------------+
|       Monthly filer|
|          Terminated|
|     Quarterly filer|
|Administratively ...|
+--------------------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cmt_mast_file=cmt_mast_file.withColumn('ORG_TP', 
      when((cmt_mast_file.ORG_TP == 'C'),regexp_replace(cmt_mast_file.ORG_TP,'C','Corporation')) \
     .when((cmt_mast_file.ORG_TP == 'L'),regexp_replace(cmt_mast_file.ORG_TP,'L','Labor organization')) \
     .when((cmt_mast_file.ORG_TP == 'M'),regexp_replace(cmt_mast_file.ORG_TP,'M','Membership organization')) \
     .when((cmt_mast_file.ORG_TP == 'T'),regexp_replace(cmt_mast_file.ORG_TP,'T','Trade association')) \
     .when((cmt_mast_file.ORG_TP == 'V'),regexp_replace(cmt_mast_file.ORG_TP,'V','Cooperative')) \
     .when((cmt_mast_file.ORG_TP == 'W'),regexp_replace(cmt_mast_file.ORG_TP,'W','Corporation without capital stock'))) 

cmt_mast_file.select("ORG_TP").distinct().show()

+--------------------+
|              ORG_TP|
+--------------------+
|                null|
|  Labor organization|
|         Cooperative|
|         Corporation|
|Membership organi...|
|Corporation witho...|
|   Trade association|
+--------------------+



In [0]:
cmt_mast_file.where(cmt_mast_file.CMTE_FILING_FREQ.contains('Quarterly filer')).count()

Out[52]: 11276

In [0]:
cmt_mast_file.write.option("header",True) \
        .partitionBy("CMTE_FILING_FREQ") \
        .mode("overwrite") \
        .parquet("/FinalProject/cmt_mast_file_Cleaned")

####Contributions from committes to candidates & independent expenditures

In [0]:
cont_and_ind_exp ="dbfs:/FinalProject/cont_and_ind_exp/"
cont_and_ind_exp_file = (spark
      .read
      .format("parquet")
      .option("path", cont_and_ind_exp)
      .load())

In [0]:
cont_and_ind_exp_file.shape()

Out[55]: (889164, 22)

In [0]:
from pyspark.sql.functions import col,isnan, when, count
cont_and_ind_exp_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cont_and_ind_exp_file.columns[:7]]).show()
cont_and_ind_exp_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cont_and_ind_exp_file.columns[8:15]]).show()
cont_and_ind_exp_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in cont_and_ind_exp_file.columns[16:22]]).show()

+-------+---------+------+---------------+---------+--------------+---------+
|CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|
+-------+---------+------+---------------+---------+--------------+---------+
|      0|        0|     0|           1868|        0|             0|     5101|
+-------+---------+------+---------------+---------+--------------+---------+

+----+-----+--------+--------+----------+--------------+---------------+
|CITY|STATE|ZIP_CODE|EMPLOYER|OCCUPATION|TRANSACTION_DT|TRANSACTION_AMT|
+----+-----+--------+--------+----------+--------------+---------------+
|1327| 1400|    1481|  889146|    889147|          5583|              0|
+----+-----+--------+--------+----------+--------------+---------------+

+-------+-------+--------+-------+---------+------+
|CAND_ID|TRAN_ID|FILE_NUM|MEMO_CD|MEMO_TEXT|SUB_ID|
+-------+-------+--------+-------+---------+------+
|   2015|    456|       0| 875931|   842592|     0|
+-------+-------+--------+-------+-

In [0]:
cs=("EMPLOYER","OCCUPATION","MEMO_CD","MEMO_TEXT")

cont_and_ind_exp_file=cont_and_ind_exp_file.drop(*cs)
len(cont_and_ind_exp_file.columns)

Out[57]: 18

In [0]:
from pyspark.sql.functions import *
cont_and_ind_exp_file = cont_and_ind_exp_file.withColumn("ZIP_CODE",cont_and_ind_exp_file["ZIP_CODE"].cast('int'))
cont_and_ind_exp_file = cont_and_ind_exp_file.withColumn('ZIP_CODE', lpad(cont_and_ind_exp_file.ZIP_CODE,5, '0'))
cont_and_ind_exp_file.select("ZIP_CODE").show()

+--------+
|ZIP_CODE|
+--------+
|   48333|
|   10022|
|   10022|
|   40201|
|   10022|
|   93389|
|   10022|
|   10022|
|   10022|
|   10022|
|   93389|
|   93290|
|   10022|
|   40201|
|   08210|
|   44021|
|   10022|
|   10022|
|   40201|
|   77469|
+--------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cont_and_ind_exp_file=cont_and_ind_exp_file.withColumn('AMNDT_IND', 
      when((cont_and_ind_exp_file.AMNDT_IND == 'N'),regexp_replace(cont_and_ind_exp_file.AMNDT_IND,'N','New')) \
     .when((cont_and_ind_exp_file.AMNDT_IND == 'A'),regexp_replace(cont_and_ind_exp_file.AMNDT_IND,'A','Amendment')) \
     .when((cont_and_ind_exp_file.AMNDT_IND == 'T'),regexp_replace(cont_and_ind_exp_file.AMNDT_IND,'T','Termination'))) 

cont_and_ind_exp_file.select("AMNDT_IND").distinct().show()

+-----------+
|  AMNDT_IND|
+-----------+
|Termination|
|        New|
|  Amendment|
+-----------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cont_and_ind_exp_file=cont_and_ind_exp_file.withColumn('TRANSACTION_PGI', 
      when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("P")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'P','Primary ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("G")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'G','General ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("O")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'O','Other ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("C")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'C','Convention ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("R")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'R','Runoff ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("S")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'S','Special ')) \
     .when((cont_and_ind_exp_file.TRANSACTION_PGI.contains("E")),regexp_replace(cont_and_ind_exp_file.TRANSACTION_PGI,'E','Recount ')))

cont_and_ind_exp_file.select("TRANSACTION_PGI").distinct().show()

+---------------+
|TRANSACTION_PGI|
+---------------+
|   General 2024|
|   General 2020|
|   Primary 2016|
|   Primary 2021|
|           null|
|   Recount 2020|
|   Special 2021|
|   Primary 2018|
|   Primary 2024|
|   General 2022|
|   General 2021|
|   Special 2022|
|   General 2014|
|   Primary 2026|
|   General 2016|
|   General 2018|
|   Primary 2020|
|       Primary |
|    Runoff 2021|
|   Recount 2022|
+---------------+
only showing top 20 rows



In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
cont_and_ind_exp_file = cont_and_ind_exp_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(IntegerType()))
cont_and_ind_exp_file.select("TRANSACTION_DT").show()

+--------------+
|TRANSACTION_DT|
+--------------+
|       9062020|
|       8032020|
|       8152020|
|       8242020|
|       7172020|
|       8032020|
|       9042020|
|       8202020|
|       7102020|
|       8172020|
|       9152020|
|       9092020|
|       9222020|
|       8262020|
|       7312020|
|       9022020|
|       9212020|
|       7022020|
|       9212020|
|       8192020|
+--------------+
only showing top 20 rows



In [0]:
from pyspark.sql.types import StringType
cont_and_ind_exp_file = cont_and_ind_exp_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(StringType()))
cont_and_ind_exp_file.select("TRANSACTION_DT").take(5)

Out[62]: [Row(TRANSACTION_DT='9062020'),
 Row(TRANSACTION_DT='8032020'),
 Row(TRANSACTION_DT='8152020'),
 Row(TRANSACTION_DT='8242020'),
 Row(TRANSACTION_DT='7172020')]

In [0]:
from pyspark.sql.functions import *

cont_and_ind_exp_file = cont_and_ind_exp_file.withColumn('TRANSACTION_DT', lpad(cont_and_ind_exp_file.TRANSACTION_DT,8, '0'))
cont_and_ind_exp_file.select("TRANSACTION_DT").take(3)

Out[63]: [Row(TRANSACTION_DT='09062020'),
 Row(TRANSACTION_DT='08032020'),
 Row(TRANSACTION_DT='08152020')]

In [0]:
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql.functions import from_unixtime, unix_timestamp
cont_and_ind_exp_file=cont_and_ind_exp_file.withColumn("TRANSACTION_DT",from_unixtime(unix_timestamp(col("TRANSACTION_DT"),'MMddyyyy'), 'MM-dd-yyyy'))
cont_and_ind_exp_file.select("TRANSACTION_DT").take(5)

Out[64]: [Row(TRANSACTION_DT='09-06-2020'),
 Row(TRANSACTION_DT='08-03-2020'),
 Row(TRANSACTION_DT='08-15-2020'),
 Row(TRANSACTION_DT='08-24-2020'),
 Row(TRANSACTION_DT='07-17-2020')]

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

cont_and_ind_exp_file=cont_and_ind_exp_file.withColumn('ENTITY_TP', 
      when((cont_and_ind_exp_file.ENTITY_TP == 'CAN'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'CAN','Candidate')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'CCM'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'CCM','Candidate Committee')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'COM'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'COM','Committee')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'IND'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'IND','Individual (a person)')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'ORG'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'ORG','Organization (not a committee and not a person)')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'PAC'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'PAC','Political Action Committee')) \
     .when((cont_and_ind_exp_file.ENTITY_TP == 'PTY'),regexp_replace(cont_and_ind_exp_file.ENTITY_TP,'PTY','Party Organization')))

cont_and_ind_exp_file.select("ENTITY_TP").distinct().show()

+--------------------+
|           ENTITY_TP|
+--------------------+
|Individual (a per...|
|Organization (not...|
|                null|
|           Committee|
| Candidate Committee|
|  Party Organization|
|           Candidate|
|Political Action ...|
+--------------------+



In [0]:
cont_and_ind_exp_file.write.option("header",True) \
        .partitionBy("ENTITY_TP") \
        .mode("overwrite") \
        .parquet("/FinalProject/cont_and_ind_exp_file_cleaned")

####House Senate current campaigns

In [0]:
House_senate ="dbfs:/FinalProject/senate"
House_senate_file = (spark
      .read
      .format("parquet")
      .option("path", House_senate)
      .load())

In [0]:
House_senate_file.shape()

Out[69]: (2639, 30)

In [0]:
from pyspark.sql.functions import col,isnan, when, count

House_senate_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in House_senate_file.columns[:10]]).show()
House_senate_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in House_senate_file.columns[11:20]]).show()
House_senate_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in House_senate_file.columns[21:30]]).show()

+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|CAND_ID|CAND_NAME|CAND_ICI|PTY_CD|CAND_PTY_AFFILIATION|TTL_RECEIPTS|TRANS_FROM_AUTH|TTL_DISB|TRANS_TO_AUTH|COH_BOP|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+
|      0|        0|       5|     0|                   0|           0|              0|       0|            0|      0|
+-------+---------+--------+------+--------------------+------------+---------------+--------+-------------+-------+

+------------+----------+-----------+---------------+----------------+-------------+-----------------+--------------+--------------------+
|CAND_CONTRIB|CAND_LOANS|OTHER_LOANS|CAND_LOAN_REPAY|OTHER_LOAN_REPAY|DEBTS_OWED_BY|TTL_INDIV_CONTRIB|CAND_OFFICE_ST|CAND_OFFICE_DISTRICT|
+------------+----------+-----------+---------------+----------------+-------------+-----------------+--------------+-------------------

In [0]:
cs=("PRIM_ELECTION","RUN_ELECTION","GEN_ELECTION","GEN_ELECTION_PRECENT")
House_senate_file=House_senate_file.drop(*cs)
len(House_senate_file.columns)

Out[72]: 26

In [0]:
from pyspark.sql.functions import when, lit
conditions = when(((House_senate_file['TRANS_FROM_AUTH'] != 0.0) & (House_senate_file['TRANS_TO_AUTH'] != 0.0)), ((House_senate_file['TTL_RECEIPTS'] - House_senate_file['TRANS_FROM_AUTH'])))
House_senate_file = House_senate_file.withColumn('TTL_RECEIPTS_CORRECTED', conditions)
House_senate_file.shape()

Out[73]: (2639, 27)

In [0]:
from pyspark.sql.functions import coalesce
House_senate_file=House_senate_file.withColumn('TTL_RECEIPTS_CORRECTED',coalesce(House_senate_file.TTL_RECEIPTS_CORRECTED,House_senate_file.TTL_RECEIPTS))
House_senate_file.select("TTL_RECEIPTS_CORRECTED").take(5)

Out[74]: [Row(TTL_RECEIPTS_CORRECTED=5253251.54),
 Row(TTL_RECEIPTS_CORRECTED=80094.95),
 Row(TTL_RECEIPTS_CORRECTED=118661.85),
 Row(TTL_RECEIPTS_CORRECTED=62935.42),
 Row(TTL_RECEIPTS_CORRECTED=2344516.53)]

In [0]:
from pyspark.sql.functions import when, lit
conditions = when(((House_senate_file['TRANS_FROM_AUTH'] != 0.0) & (House_senate_file['TRANS_TO_AUTH'] != 0.0)), ((House_senate_file['TTL_DISB'] - House_senate_file['TRANS_TO_AUTH'])))
House_senate_file = House_senate_file.withColumn('TTL_DISB_CORRECTED', conditions)

In [0]:
from pyspark.sql.functions import coalesce
House_senate_file=House_senate_file.withColumn('TTL_DISB_CORRECTED',coalesce(House_senate_file.TTL_DISB_CORRECTED,House_senate_file.TTL_DISB))
House_senate_file.select("TTL_DISB_CORRECTED").head(3)

Out[76]: [Row(TTL_DISB_CORRECTED=5162902.93),
 Row(TTL_DISB_CORRECTED=78973.24),
 Row(TTL_DISB_CORRECTED=118661.85)]

In [0]:
House_senate_file.write.option("header",True) \
        .mode("overwrite") \
        .parquet("/FinalProject/House_senate_file_cleaned")

####Any transaction from one committee to another

In [0]:
any_trans_one_cmt_to_other ="dbfs:/FinalProject/transaction_from_cmt/"
any_trans_one_cmt_to_other_file = (spark
      .read
      .format("parquet")
      .option("path", any_trans_one_cmt_to_other)
      .load())

In [0]:
any_trans_one_cmt_to_other_file.shape()

Out[80]: (7455021, 21)

In [0]:
from pyspark.sql.functions import col,isnan, when, count
any_trans_one_cmt_to_other_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in any_trans_one_cmt_to_other_file.columns[:7]]).show()
any_trans_one_cmt_to_other_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in any_trans_one_cmt_to_other_file.columns[8:14]]).show()
any_trans_one_cmt_to_other_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in any_trans_one_cmt_to_other_file.columns[15:21]]).show()

+-------+---------+------+---------------+---------+--------------+---------+
|CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|
+-------+---------+------+---------------+---------+--------------+---------+
|      0|        0|     0|           1969|        0|             0|     5734|
+-------+---------+------+---------------+---------+--------------+---------+

+----+-----+--------+--------+----------+--------------+
|CITY|STATE|ZIP_CODE|EMPLOYER|OCCUPATION|TRANSACTION_DT|
+----+-----+--------+--------+----------+--------------+
|2000| 2392|    2421| 1761531|   1761590|          5583|
+----+-----+--------+--------+----------+--------------+

+--------+-------+--------+-------+---------+------+
|OTHER_ID|TRAN_ID|FILE_NUM|MEMO_CD|MEMO_TEXT|SUB_ID|
+--------+-------+--------+-------+---------+------+
| 5696788|    456|       0|1734350|  2761476|     0|
+--------+-------+--------+-------+---------+------+



In [0]:
any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.drop("OTHER_ID")
len(any_trans_one_cmt_to_other_file.columns)

Out[82]: 20

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.withColumn('AMNDT_IND', 
     when((any_trans_one_cmt_to_other_file.AMNDT_IND == 'N'),regexp_replace(any_trans_one_cmt_to_other_file.AMNDT_IND,'N','New')) \
     .when((any_trans_one_cmt_to_other_file.AMNDT_IND == 'A'),regexp_replace(any_trans_one_cmt_to_other_file.AMNDT_IND,'A','Amendment')) \
     .when((any_trans_one_cmt_to_other_file.AMNDT_IND == 'T'),regexp_replace(any_trans_one_cmt_to_other_file.AMNDT_IND,'T','Termination'))) 
any_trans_one_cmt_to_other_file.select("AMNDT_IND").distinct().show()

+-----------+
|  AMNDT_IND|
+-----------+
|        New|
|  Amendment|
|Termination|
+-----------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.withColumn('TRANSACTION_PGI', 
      when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("P")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'P','Primary ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("G")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'G','General ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("O")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'O','Other ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("C")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'C','Convention ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("R")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'R','Runoff ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("S")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'S','Special ')) \
     .when((any_trans_one_cmt_to_other_file.TRANSACTION_PGI.contains("E")),regexp_replace(any_trans_one_cmt_to_other_file.TRANSACTION_PGI,'E','Recount ')))
any_trans_one_cmt_to_other_file.select("TRANSACTION_PGI").distinct().show()

+---------------+
|TRANSACTION_PGI|
+---------------+
|   General 2024|
|   General 2020|
|   General 2019|
|   Recount 2020|
|   Primary 2024|
|   General 2022|
|   Primary 2014|
|   Primary 2026|
|   Primary 2020|
|       Primary |
|   Special 2020|
|   Primary 2022|
|    Runoff 2020|
|   Primary 2002|
|Convention 2018|
|   Primary 2016|
|           null|
|   Primary 2018|
|   Special 2019|
|Convention 2020|
+---------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.withColumn('ENTITY_TP', 
      when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'CAN'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'CAN','Candidate')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'CCM'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'CCM','Candidate Committee')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'COM'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'COM','Committee')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'IND'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'IND','Individual (a person)')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'ORG'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'ORG','Organization (not a committee and not a person)')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'PAC'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'PAC','Political Action Committee')) \
     .when((any_trans_one_cmt_to_other_file.ENTITY_TP == 'PTY'),regexp_replace(any_trans_one_cmt_to_other_file.ENTITY_TP,'PTY','Party Organization')))

any_trans_one_cmt_to_other_file.select("ENTITY_TP").distinct().show()

+--------------------+
|           ENTITY_TP|
+--------------------+
|Individual (a per...|
|Organization (not...|
|           Committee|
|Political Action ...|
|                null|
| Candidate Committee|
|  Party Organization|
|           Candidate|
+--------------------+



In [0]:
from pyspark.sql.functions import *
any_trans_one_cmt_to_other_file = any_trans_one_cmt_to_other_file.withColumn("ZIP_CODE",any_trans_one_cmt_to_other_file["ZIP_CODE"].cast('int'))
any_trans_one_cmt_to_other_file = any_trans_one_cmt_to_other_file.withColumn('ZIP_CODE', lpad(any_trans_one_cmt_to_other_file.ZIP_CODE,5, '0'))
any_trans_one_cmt_to_other_file.select("ZIP_CODE").show()

+--------+
|ZIP_CODE|
+--------+
|   16803|
|   33917|
|   54523|
|   80918|
|   51360|
|   94306|
|   75218|
|   95959|
|   77332|
|   20720|
|   20720|
|   91205|
|   18705|
|   22205|
|   94066|
|   30306|
|   74074|
|   37421|
|   76310|
|   76310|
+--------+
only showing top 20 rows



In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import *
any_trans_one_cmt_to_other_file = any_trans_one_cmt_to_other_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(IntegerType()))
any_trans_one_cmt_to_other_file = any_trans_one_cmt_to_other_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(StringType()))
any_trans_one_cmt_to_other_file = any_trans_one_cmt_to_other_file.withColumn('TRANSACTION_DT', lpad(any_trans_one_cmt_to_other_file.TRANSACTION_DT,8, '0'))
any_trans_one_cmt_to_other_file.select("TRANSACTION_DT").take(3)

Out[88]: [Row(TRANSACTION_DT='10062020'),
 Row(TRANSACTION_DT='09242020'),
 Row(TRANSACTION_DT='10062020')]

In [0]:
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql.functions import from_unixtime, unix_timestamp
any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.withColumn("TRANSACTION_DT",from_unixtime(unix_timestamp(col("TRANSACTION_DT"),'MMddyyyy'), 'MM-dd-yyyy'))
any_trans_one_cmt_to_other_file.select("TRANSACTION_DT").take(5)

Out[89]: [Row(TRANSACTION_DT='10-06-2020'),
 Row(TRANSACTION_DT='09-24-2020'),
 Row(TRANSACTION_DT='10-06-2020'),
 Row(TRANSACTION_DT='10-06-2020'),
 Row(TRANSACTION_DT='10-08-2020')]

In [0]:
any_trans_one_cmt_to_other_file=any_trans_one_cmt_to_other_file.drop("MEMO_CD","MEMO_TEXT")
len(any_trans_one_cmt_to_other_file.columns)

Out[90]: 18

In [0]:
any_trans_one_cmt_to_other_file.write.option("header",True) \
        .mode("overwrite") \
        .parquet("/FinalProject/any_trans_one_cmt_to_other_file_cleaned")

####Contributions by individuals

In [0]:
Cnt_by_individuals ="dbfs:/FinalProject/Cont_by_indv/"
Cnt_by_individuals_file = (spark
      .read
      .format("parquet")
      .option("path", Cnt_by_individuals)
      .load())

In [0]:
Cnt_by_individuals_file.shape()

Out[6]: (26170486, 21)

In [0]:
from pyspark.sql.functions import col,isnan, when, count

Cnt_by_individuals_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in Cnt_by_individuals_file.columns[:8]]).show()
Cnt_by_individuals_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in Cnt_by_individuals_file.columns[8:16]]).show()
Cnt_by_individuals_file.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in Cnt_by_individuals_file.columns[16:21]]).show()

+-------+---------+------+---------------+---------+--------------+---------+----+
|CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|NAME|
+-------+---------+------+---------------+---------+--------------+---------+----+
|      0|        0|  7776|        6316052|     8713|          8719|     8779|8725|
+-------+---------+------+---------------+---------+--------------+---------+----+

+-----+-----+--------+--------+----------+--------------+---------------+--------+
| CITY|STATE|ZIP_CODE|EMPLOYER|OCCUPATION|TRANSACTION_DT|TRANSACTION_AMT|OTHER_ID|
+-----+-----+--------+--------+----------+--------------+---------------+--------+
|11284|17295|   27768| 1395258|    807962|          8741|           8731| 7891858|
+-----+-----+--------+--------+----------+--------------+---------------+--------+

+-------+--------+--------+---------+------+
|TRAN_ID|FILE_NUM| MEMO_CD|MEMO_TEXT|SUB_ID|
+-------+--------+--------+---------+------+
|   8732|    8731|26093334|  7816

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

Cnt_by_individuals_file=Cnt_by_individuals_file.withColumn('AMNDT_IND', 
      when((Cnt_by_individuals_file.AMNDT_IND == 'N'),regexp_replace(Cnt_by_individuals_file.AMNDT_IND,'N','New')) \
     .when((Cnt_by_individuals_file.AMNDT_IND == 'A'),regexp_replace(Cnt_by_individuals_file.AMNDT_IND,'A','Amendment')) \
     .when((Cnt_by_individuals_file.AMNDT_IND == 'T'),regexp_replace(Cnt_by_individuals_file.AMNDT_IND,'T','Termination'))) 

Cnt_by_individuals_file.select("AMNDT_IND").distinct().show()

+-----------+
|  AMNDT_IND|
+-----------+
|Termination|
|        New|
|  Amendment|
|       null|
+-----------+



In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

Cnt_by_individuals_file=Cnt_by_individuals_file.withColumn('TRANSACTION_PGI', 
      when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("P")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'P','Primary ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("G")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'G','General ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("O")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'O','Other ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("C")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'C','Convention ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("R")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'R','Runoff ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("S")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'S','Special ')) \
     .when((Cnt_by_individuals_file.TRANSACTION_PGI.contains("E")),regexp_replace(Cnt_by_individuals_file.TRANSACTION_PGI,'E','Recount ')))

In [0]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

Cnt_by_individuals_file=Cnt_by_individuals_file.withColumn('ENTITY_TP', 
      when((Cnt_by_individuals_file.ENTITY_TP == 'CAN'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'CAN','Candidate')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'CCM'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'CCM','Candidate Committee')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'COM'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'COM','Committee')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'IND'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'IND','Individual (a person)')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'ORG'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'ORG','Organization (not a committee and not a person)')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'PAC'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'PAC','Political Action Committee')) \
     .when((Cnt_by_individuals_file.ENTITY_TP == 'PTY'),regexp_replace(Cnt_by_individuals_file.ENTITY_TP,'PTY','Party Organization')))

Cnt_by_individuals_file.select("ENTITY_TP").distinct().show()

+--------------------+
|           ENTITY_TP|
+--------------------+
|Individual (a per...|
|Organization (not...|
|                null|
|           Committee|
| Candidate Committee|
|  Party Organization|
|           Candidate|
|Political Action ...|
+--------------------+



In [0]:
from pyspark.sql.functions import *
Cnt_by_individuals_file = Cnt_by_individuals_file.withColumn("ZIP_CODE",Cnt_by_individuals_file["ZIP_CODE"].cast('int'))
Cnt_by_individuals_file = Cnt_by_individuals_file.withColumn('ZIP_CODE', lpad(Cnt_by_individuals_file.ZIP_CODE,5, '0'))
Cnt_by_individuals_file.select("ZIP_CODE").show()

+--------+
|ZIP_CODE|
+--------+
|   85372|
|   85372|
|   78130|
|   95519|
|   90046|
|   90016|
|   19143|
|   33076|
|   75150|
|   27514|
|   33308|
|   95519|
|   48306|
|   80301|
|   38138|
|   89048|
|   30022|
|   04011|
|   10022|
|   10022|
+--------+
only showing top 20 rows



In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import *

Cnt_by_individuals_file = Cnt_by_individuals_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(IntegerType()))
Cnt_by_individuals_file = Cnt_by_individuals_file.withColumn("TRANSACTION_DT",col("TRANSACTION_DT").cast(StringType()))
Cnt_by_individuals_file = Cnt_by_individuals_file.withColumn('TRANSACTION_DT', lpad(Cnt_by_individuals_file.TRANSACTION_DT,8, '0'))
Cnt_by_individuals_file.select("TRANSACTION_DT").take(3)

Out[13]: [Row(TRANSACTION_DT='09192019'),
 Row(TRANSACTION_DT='09192019'),
 Row(TRANSACTION_DT='07072019')]

In [0]:
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql.functions import from_unixtime, unix_timestamp
Cnt_by_individuals_file=Cnt_by_individuals_file.withColumn("TRANSACTION_DT",from_unixtime(unix_timestamp(col("TRANSACTION_DT"),'MMddyyyy'), 'MM-dd-yyyy'))
Cnt_by_individuals_file.select("TRANSACTION_DT").take(5)

Out[14]: [Row(TRANSACTION_DT='09-19-2019'),
 Row(TRANSACTION_DT='09-19-2019'),
 Row(TRANSACTION_DT='07-07-2019'),
 Row(TRANSACTION_DT='11-18-2019'),
 Row(TRANSACTION_DT='08-29-2019')]

In [0]:
Cnt_by_individuals_file.write.option("header",True) \
        .partitionBy("STATE") \
        .mode("overwrite") \
        .parquet("/FinalProject/Cnt_by_individuals_file_cleaned")

In [0]:
%fs ls "/FinalProject"

path,name,size,modificationTime
dbfs:/FinalProject/All_Candidates_Cleaned/,All_Candidates_Cleaned/,0,0
dbfs:/FinalProject/CCL_file_Cleaned/,CCL_file_Cleaned/,0,0
dbfs:/FinalProject/Candidate_Master_Cleaned/,Candidate_Master_Cleaned/,0,0
dbfs:/FinalProject/Cnt_by_individuals_file_cleaned/,Cnt_by_individuals_file_cleaned/,0,0
dbfs:/FinalProject/Cont_by_indv/,Cont_by_indv/,0,0
dbfs:/FinalProject/House_senate_file_cleaned/,House_senate_file_cleaned/,0,0
dbfs:/FinalProject/PAC/,PAC/,0,0
dbfs:/FinalProject/PAC_Cleaned/,PAC_Cleaned/,0,0
dbfs:/FinalProject/all/,all/,0,0
dbfs:/FinalProject/any_trans_one_cmt_to_other_file_cleaned/,any_trans_one_cmt_to_other_file_cleaned/,0,0
